In [6]:
!pip install pyspark


     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 49.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1201a4fbc26139c725d9c466c3e825f4bc4503bbac0d5343d4cb24f685f0ee0d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [31]:
import os
import urllib.request
import numpy as np

In [32]:


#urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip')
!unzip "/content/glove.6B.zip"





Archive:  /content/glove.6B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/glove.6B.zip or
        /content/glove.6B.zip.zip, and cannot find /content/glove.6B.zip.ZIP, period.


In [33]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf,lower
from pyspark.sql.types import StringType,IntegerType,FloatType,LongType,BooleanType,TimestampType
from pyspark.sql.types import StructType
spark = SparkSession.builder\
        .master("local")\
        .appName("RECOMMENDATION SYSTEM")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


spark

In [ ]:
schema = StructType() \
      .add("Course_ID",IntegerType(),True) \
      .add("Course Title",StringType(),True) \
      .add("Url",StringType(),True) \
      .add("Is_paid",BooleanType(),True) \
      .add("Price",IntegerType(),True) \
      .add("Num Subscriber",IntegerType(),True) \
      .add("Num Reviewers",IntegerType(),True) \
      .add("Num Lectures",IntegerType(),True) \
      .add("Level",StringType(),True) \
      .add("Content Duration",FloatType(),True) \
      .add("Publish Timestamp",TimestampType(),True) \
      .add("Subjects",StringType(),True) 
      


from google.colab import files

uploaded = files.upload()
   


TypeError: ignored

In [12]:
df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("/content/udemy_courses.csv")


df_with_schema.show()

+---------+--------------------+--------------------+-------+-----+--------------+-------------+------------+------------------+----------------+-------------------+----------------+
|Course_ID|        Course Title|                 Url|Is_paid|Price|Num Subscriber|Num Reviewers|Num Lectures|             Level|Content Duration|  Publish Timestamp|        Subjects|
+---------+--------------------+--------------------+-------+-----+--------------+-------------+------------+------------------+----------------+-------------------+----------------+
|  1070968|Ultimate Investme...|https://www.udemy...|   true|  200|          2147|           23|          51|        All Levels|             1.5|2017-01-18 20:58:58|Business Finance|
|  1113822|Complete GST Cour...|https://www.udemy...|   true|   75|          2792|          923|         274|        All Levels|            39.0|2017-03-09 16:34:20|Business Finance|
|  1006314|Financial Modelin...|https://www.udemy...|   true|   45|          2174|   

In [13]:
df_Course_Title=df_with_schema.select(col("Course Title"))
df_Course_Sector=df_with_schema.select(col("Subjects"))

In [14]:
from pyspark.sql.functions import udf

import string
regular_punct = list(string.punctuation)
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤','⌨','☝']

all_punct = list(set(regular_punct + extra_punct))

def remove_punctuation(text):
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, ' ')
    return (text.strip().lower())
    

DF_string_remover = udf(lambda m: remove_punctuation(m))

df_Course_Title=df_Course_Title.withColumn("Course Title",DF_string_remover("Course Title"))
df_Course_Sector=df_Course_Sector.withColumn("Subjects",DF_string_remover("Subjects"))


In [15]:
df_Course_Title.show()

+--------------------+
|        Course Title|
+--------------------+
|ultimate investme...|
|complete gst cour...|
|financial modelin...|
|beginner to pro  ...|
|how to maximize y...|
|trading penny sto...|
|investing and tra...|
|trading stock cha...|
|options trading 3...|
|the only investme...|
|forex trading sec...|
|trading options w...|
|financial managem...|
|forex trading cou...|
|python algo tradi...|
|short selling  le...|
|basic technical a...|
|the complete char...|
|7 deadly mistakes...|
|financial stateme...|
+--------------------+
only showing top 20 rows



In [16]:
df_Course_Sector.show(truncate=False)

+----------------+
|Subjects        |
+----------------+
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
|business finance|
+----------------+
only showing top 20 rows



In [17]:
from pyspark.ml.feature import Tokenizer

tokenizer1 = Tokenizer(inputCol="Course Title", outputCol="Course Title_tokenise")
df_Title_tokenise= tokenizer1.transform(df_Course_Title).select("Course Title_tokenise")

tokenizer = Tokenizer(inputCol="Subjects", outputCol="Course Subjects_tokenise")
df_subjects_tokenise= tokenizer.transform(df_Course_Sector).select("Course Subjects_tokenise")

In [18]:
df_subjects_tokenise.show()
df_Title_tokenise.show()


+------------------------+
|Course Subjects_tokenise|
+------------------------+
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
|     [business, finance]|
+------------------------+
only showing top 20 rows

+---------------------+
|Course Title_tokenise|
+---------------------+
| [ultimate, invest...|
| [complete, gst, c...|
| [financial, model...|
| [beginner, to, pr...|
| [how, to, maximiz...|
| [trading, penny, ...|
| [investing, and, ...|
| [trading, stock, ...|
| [options, trading...|
| [the, only, inves...|
| [forex, trad

In [19]:
from pyspark.ml.feature import StopWordsRemover


remover1 = StopWordsRemover()
stopwords = remover1.getStopWords()
remover1.setInputCol("Course Subjects_tokenise")
remover1.setOutputCol("Course subjects_tokenise_no_stopw")
Course_subjects_no_stopw_df = remover1.transform(df_subjects_tokenise).select("Course subjects_tokenise_no_stopw")



remover2 = StopWordsRemover()
stopwords = remover2.getStopWords()
remover2.setInputCol("Course Title_tokenise")
remover2.setOutputCol("Course Title_tokenise_no_stopw")
df_Title_tokenise_no_stopw_df  = remover2.transform(df_Title_tokenise).select("Course Title_tokenise_no_stopw")

In [20]:
df_Title_tokenise_no_stopw_df.columns

['Course Title_tokenise_no_stopw']

In [21]:
Course_subjects_no_stopw_df.columns

['Course subjects_tokenise_no_stopw']

In [22]:
#delete the numeric and check spaces in Course title

def mannual_sanitisation(text):
  text_sanitised=[]
  for i in text:
    if len(i)>1 and i != " ":
      if i.isalpha():
        text_sanitised.append(i)

  return(text_sanitised)

DF_sanitizer = udf(lambda m: mannual_sanitisation(m))

df_Title_tokenise_no_stopw_df=df_Title_tokenise_no_stopw_df.withColumn("Course Title_tokenise_no_stopw",DF_sanitizer("Course Title_tokenise_no_stopw"))
df_Title_tokenise_no_stopw_df.show()         
      

+------------------------------+
|Course Title_tokenise_no_stopw|
+------------------------------+
|          [ultimate, invest...|
|          [complete, gst, c...|
|          [financial, model...|
|          [beginner, pro, f...|
|          [maximize, profit...|
|          [trading, penny, ...|
|          [investing, tradi...|
|          [trading, stock, ...|
|          [options, trading...|
|          [investment, stra...|
|          [forex, trading, ...|
|          [trading, options...|
|          [financial, manag...|
|          [forex, trading, ...|
|          [python, algo, tr...|
|          [short, selling, ...|
|          [basic, technical...|
|          [complete, chart,...|
|          [deadly, mistakes...|
|          [financial, state...|
+------------------------------+
only showing top 20 rows



In [23]:
df_Title_tokenise_no_stopw_df.show() 

+------------------------------+
|Course Title_tokenise_no_stopw|
+------------------------------+
|          [ultimate, invest...|
|          [complete, gst, c...|
|          [financial, model...|
|          [beginner, pro, f...|
|          [maximize, profit...|
|          [trading, penny, ...|
|          [investing, tradi...|
|          [trading, stock, ...|
|          [options, trading...|
|          [investment, stra...|
|          [forex, trading, ...|
|          [trading, options...|
|          [financial, manag...|
|          [forex, trading, ...|
|          [python, algo, tr...|
|          [short, selling, ...|
|          [basic, technical...|
|          [complete, chart,...|
|          [deadly, mistakes...|
|          [financial, state...|
+------------------------------+
only showing top 20 rows



In [24]:
Course_subjects_no_stopw_df.columns

['Course subjects_tokenise_no_stopw']

In [ ]:
import numpy as np
emmbed_dict = {}
with open('/content/glove.6B.50d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector

'Course subjects_tokenise_no_stopw'

'Course subjects_tokenise_no_stopw'

In [ ]:
def vectorise_words(text):
  vectorise_words=[]
  for i in text:
    try:
        vect=emmbed_dict[i]
        for i in range(0,5):
          vectorise_words.append(vect[i])
    except:
        for i in range(0,5):
           vectorise_words.append(0)

  return (vectorise_words)
        
DF_word_vec_calculator= udf(lambda m: vectorise_words(m))


pandasDF_title = df_Title_tokenise_no_stopw_df.toPandas()
print(pandasDF_title)

NameError: ignored

In [ ]:
import time
vectorise_doc=[]
for title in pandasDF_title["Course Title_tokenise_no_stopw"]:
  print("Title::::",title)
  vectorise_doc.append(vectorise_words(title))
  
  

Title:::: ['ultimate', 'investment', 'banking', 'course']
Title:::: ['complete', 'gst', 'course', '', '', 'certification', '', '', 'grow', 'ca', 'practice']
Title:::: ['financial', 'modeling', 'business', 'analysts', 'consultants']
Title:::: ['beginner', 'pro', '', '', 'financial', 'analysis', 'excel', '2017']
Title:::: ['maximize', 'profits', 'trading', 'options']
Title:::: ['trading', 'penny', 'stocks', '', 'guide', 'levels', '2017']
Title:::: ['investing', 'trading', 'beginners', '', 'mastering', 'price', 'charts']
Title:::: ['trading', 'stock', 'chart', 'patterns', 'immediate', '', 'explosive', 'gains']
Title:::: ['options', 'trading', '3', '', '', 'advanced', 'stock', 'profit', 'success', 'method']
Title:::: ['investment', 'strategy', 'need', 'retirement']
Title:::: ['forex', 'trading', 'secrets', 'pros', 'amazon', 'aws']
Title:::: ['trading', 'options', 'money', 'flow']
Title:::: ['financial', 'management', 'risk', 'return', 'securities']
Title:::: ['forex', 'trading', 'course', 

In [ ]:
#type(vectorise_doc)
array_vectorise_doc=np.asarray(vectorise_doc)
type(array_vectorise_doc)
array_vectorise_doc.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


(3683,)

In [ ]:
vectorise_title=[]
for i,rows in enumerate(array_vectorise_doc):
  print(i,rows)
  for j,c in enumerate(array_vectorise_doc)""

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
np.savetxt('Course_title_vectorised.txt', array_vectorise_doc)

TypeError: ignored

In [ ]:
import time
for title in pandasDF_title["Course Title_tokenise_no_stopw"]:
  print("Title::::",title)
  
  vect_word=[]
  for words in title:
    if len(words) >1:
        print("Words:::::",words)
        time.sleep(1)
         
        try:
            vec=emmbed_dict[words]
            print("Word to vector:::",vec)
            
            for i in vec:
                vect_word.append(i)

        except:
          print("Adding zeros........")
          
          for i in range(0,50):
              vect_word.append(0)